In [2]:
import requests
import json
import re
import time

PUSHSHIFT_REDDIT_URL = "http://api.pushshift.io/reddit"

def fetchObjects(subreddit, typ, after_date):

    # Perform an API request
    r = requests.get(PUSHSHIFT_REDDIT_URL + "/" + str(typ) + "/search/" + "?subreddit=" + 
                     str(subreddit) + "&sort=asc&ort_type=created_utc&after=" + str(after_date) +
                     "&size=1000",timeout=30)

    # Check the status code, if successful, process the data
    if r.status_code == 200:
        response = json.loads(r.text)
        data = response['data']
        sorted_data_by_id = sorted(data, key=lambda x: int(x['id'],36))
        return sorted_data_by_id

def extract_reddit_data(subreddit, typ):
    # Speficify the start timestamp
    print(subreddit)
    print(typ)
    max_created_utc = 1119484800 #06/23/2005 @ 12:00 AM 1356998400  # 01/01/2013 @ 12:00am (UTC)
    max_id = 0

    # Open a file for JSON output
    file = open(subreddit + "_" + typ + "April22.json","a")

    # While loop for recursive function
    while 1:
        nothing_processed = True
        # Call the recursive function
        objects = fetchObjects(subreddit, typ, max_created_utc)
        
        # Loop the returned data, ordered by date
        for object in objects:
            id = int(object['id'],36)
            if id > max_id:
                nothing_processed = False
                created_utc = object['created_utc']
                max_id = id
                if created_utc > max_created_utc: max_created_utc = created_utc
                # Output JSON data to the opened file
                print(json.dumps(object,sort_keys=True,ensure_ascii=True),file=file)
        
        # Exit if nothing happened
        if nothing_processed: return
        max_created_utc -= 1

        # Sleep a little before the next recursive function call
        time.sleep(.5)

In [3]:
extract_reddit_data("aita", "submission")
extract_reddit_data("aita", "comment")

aita
submission
aita
comment
